import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization,Input,ReLU,add,MaxPool1D,GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks,Model,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=80          #80
cls_num=5
btch=20
for i in range (0,cls_num):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=ProximalPhalanxOutlineCorrect0(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg0)
print(0,len(vars()['ecg'+str(0)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(0)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (100, 1025)
first_test_shape= (2425, 1025)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [20 20 20 20 20]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -2.8941765
first_train_sample=
 [1.        1.1309687 1.1204832 ... 1.0931087 1.1025334 1.1120469]
1 20	2 20	3 20	4 20	5 20	

In [3]:
np.shape(ecg)

(100, 1025)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 20	2 20	3 20	4 20	5 20	
 max = 20


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (0,cls_num):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (20, 1025)
Up to class  2 train shape =  (40, 1025)
Up to class  3 train shape =  (60, 1025)
Up to class  4 train shape =  (80, 1025)
Up to class  5 train shape =  (100, 1025)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 20 20 20 20 20]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (0,cls_num):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (0,cls_num):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  3.2147954000000003
min magnitude class 1  =  -2.8853939
after normalizing >>
max magnitude class 1  =  1.1640359935145925
min magnitude class 1  =  -0.9968888607072098

 cls 2  >> 
max magnitude class 2  =  2.1563458
min magnitude class 2  =  -3.1616234000000003
after normalizing >>
max magnitude class 2  =  0.7890918835701477
min magnitude class 2  =  -1.0947401178836489

 cls 3  >> 
max magnitude class 3  =  3.2339357999999994
min magnitude class 3  =  -2.679459
after normalizing >>
max magnitude class 3  =  1.1708162693463189
min magnitude class 3  =  -0.9239386911507206

 cls 4  >> 
max magnitude class 4  =  2.9995496
min magnitude class 4  =  -3.2768523000000007
after normalizing >>
max magnitude class 4  =  1.0877875402105524
min magnitude class 4  =  -1.1355586862409686

 cls 5  >> 
max magnitude class 5  =  2.7769078
min magnitude class 5  =  -2.6748398
after normalizing >>
max magnitude class 5  =  1.0089191353876594
min magnitude class 5

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.1355586862409686
max tst = 1.1708162693463189


In [14]:
len(wndws_test)

2425

In [15]:
np.shape(wndws_test)

(2425, 1025)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(20, 1025)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (100, 1024)
y_train =>  (100,)
X_test  =>  (2425, 1024)
y_test  =>  (2425,)
X_valid  =>  (100, 1024)
y_valid  =>  (100,)


In [22]:
if min(y_train)==0:
    print(' class was started from 0=Zero ')
else:
    print('!!!!!! class Should start from 0=Zero !!!!!!')
    print("!!!!!!   Reduce 1 From Class Number    !!!!!")
    y_train=np.array(y_train-1)
    y_test=np.array(y_test-1)
    y_valid=np.array(y_valid-1)
print('final Class Lables=',set(y_train))

!!!!!! class Should start from 0=Zero !!!!!!
!!!!!!   Reduce 1 From Class Number    !!!!!
final Class Lables= {0, 1, 2, 3, 4}


In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

<h1><center><div style="direction:rtl;font-family:B Nazanin">Network</div></center></h1>

In [38]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()

#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(n_timesteps,n_features)))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
5/5 [==============================] - 3s 567ms/step - loss: 1.6014 - accuracy: 0.3500 - val_loss: 1.4064 - val_accuracy: 0.7100
Epoch 2/1000
5/5 [==============================] - 0s 17ms/step - loss: 1.2548 - accuracy: 0.5300 - val_loss: 0.9175 - val_accuracy: 0.6600
Epoch 3/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.7717 - accuracy: 0.7000 - val_loss: 0.5575 - val_accuracy: 0.7400
Epoch 4/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.6816 - accuracy: 0.7500 - val_loss: 0.5291 - val_accuracy: 0.9000
Epoch 5/1000
5/5 [==============================] - 0s 17ms/step - loss: 0.5325 - accuracy: 0.8000 - val_loss: 0.4922 - val_accuracy: 0.7700
Epoch 6/1000
5/5 [==============================] - 0s 18ms/step - loss: 0.4249 - accuracy: 0.8400 - val_loss: 0.4777 - val_accuracy: 0.8000
Epoch 7/1000
5/5 [==============================] - 0s 17ms/step - loss: 0.4226 - accuracy: 0.8000 - val_loss: 0.3460 - val_accuracy: 0.8500
Epoch 8/1000

In [39]:
#model_crs.summary()
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

76/76 [==============================] - 0s 3ms/step
array([[392,  17,   9,  15,  25],
       [ 15, 615,   0,  12,  12],
       [ 10,   6, 357,  41,   4],
       [ 33,  21, 166, 350,  11],
       [  0,   3,   1,   1, 309]], dtype=int64)
              precision    recall  f1-score   support

           0       0.87      0.86      0.86       458
           1       0.93      0.94      0.93       654
           2       0.67      0.85      0.75       418
           3       0.84      0.60      0.70       581
           4       0.86      0.98      0.92       314

    accuracy                           0.83      2425
   macro avg       0.83      0.85      0.83      2425
weighted avg       0.84      0.83      0.83      2425



In [24]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()

#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(n_timesteps,n_features)))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=2048, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
5/5 [==============================] - 9s 91ms/step - loss: 5.8714 - accuracy: 0.1400 - val_loss: 1.6042 - val_accuracy: 0.2100
Epoch 2/1000
5/5 [==============================] - 0s 41ms/step - loss: 1.6005 - accuracy: 0.2500 - val_loss: 1.5069 - val_accuracy: 0.5500
Epoch 3/1000
5/5 [==============================] - 0s 42ms/step - loss: 1.3485 - accuracy: 0.5500 - val_loss: 1.1821 - val_accuracy: 0.6500
Epoch 4/1000
5/5 [==============================] - 0s 32ms/step - loss: 0.9264 - accuracy: 0.6600 - val_loss: 1.2565 - val_accuracy: 0.4600
Epoch 5/1000
5/5 [==============================] - 2s 554ms/step - loss: 0.7669 - accuracy: 0.7200 - val_loss: 0.5823 - val_accuracy: 0.7300
Epoch 6/1000
5/5 [==============================] - 0s 40ms/step - loss: 0.5881 - accuracy: 0.7400 - val_loss: 0.4461 - val_accuracy: 0.8200
Epoch 7/1000
5/5 [==============================] - 0s 31ms/step - loss: 0.4991 - accuracy: 0.7900 - val_loss: 0.4009 - val_accuracy: 0.8100
Epoch 8/1000

Epoch 59/1000
5/5 [==============================] - 0s 5ms/step - loss: 0.0311 - accuracy: 1.0000
np.shape(X_test)= (2425, 1024, 1)
Accuracy=  1.0


In [25]:
#model_crs.summary()
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

76/76 [==============================] - 1s 5ms/step
array([[384,   7,   9,  20,  38],
       [  2, 603,   0,  49,   0],
       [ 10,   7, 334,  65,   2],
       [ 19,  10, 108, 432,  12],
       [  0,   3,   1,   3, 307]], dtype=int64)
              precision    recall  f1-score   support

           0       0.93      0.84      0.88       458
           1       0.96      0.92      0.94       654
           2       0.74      0.80      0.77       418
           3       0.76      0.74      0.75       581
           4       0.86      0.98      0.91       314

    accuracy                           0.85      2425
   macro avg       0.85      0.86      0.85      2425
weighted avg       0.85      0.85      0.85      2425



In [36]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()

#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=128, kernel_size=6, activation='relu', input_shape=(n_timesteps,n_features)))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=2048, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
5/5 [==============================] - 1s 75ms/step - loss: 3.6598 - accuracy: 0.1800 - val_loss: 1.6019 - val_accuracy: 0.2000
Epoch 2/1000
5/5 [==============================] - 2s 537ms/step - loss: 1.5414 - accuracy: 0.3100 - val_loss: 1.3837 - val_accuracy: 0.5500
Epoch 3/1000
5/5 [==============================] - 0s 28ms/step - loss: 1.5670 - accuracy: 0.5100 - val_loss: 1.2423 - val_accuracy: 0.5400
Epoch 4/1000
5/5 [==============================] - 0s 33ms/step - loss: 1.0281 - accuracy: 0.6200 - val_loss: 0.9492 - val_accuracy: 0.5900
Epoch 5/1000
5/5 [==============================] - 0s 34ms/step - loss: 0.8103 - accuracy: 0.6400 - val_loss: 0.7047 - val_accuracy: 0.7000
Epoch 6/1000
5/5 [==============================] - 0s 34ms/step - loss: 0.6875 - accuracy: 0.7400 - val_loss: 0.5575 - val_accuracy: 0.7700
Epoch 7/1000
5/5 [==============================] - 0s 27ms/step - loss: 0.6240 - accuracy: 0.7800 - val_loss: 0.6751 - val_accuracy: 0.7700
Epoch 8/1000

In [37]:
#model_crs.summary()
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

76/76 [==============================] - 0s 5ms/step
array([[387,  12,   7,  14,  38],
       [ 13, 607,   0,  34,   0],
       [ 11,   9, 293,  99,   6],
       [ 19,  28,  94, 428,  12],
       [  0,   5,   0,   3, 306]], dtype=int64)
              precision    recall  f1-score   support

           0       0.90      0.84      0.87       458
           1       0.92      0.93      0.92       654
           2       0.74      0.70      0.72       418
           3       0.74      0.74      0.74       581
           4       0.85      0.97      0.91       314

    accuracy                           0.83      2425
   macro avg       0.83      0.84      0.83      2425
weighted avg       0.83      0.83      0.83      2425



In [28]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()

#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=1024, kernel_size=24, activation='relu', input_shape=(n_timesteps,n_features)))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=512, kernel_size=12, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=2048, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
5/5 [==============================] - 3s 134ms/step - loss: 632.3868 - accuracy: 0.1700 - val_loss: 1.7121 - val_accuracy: 0.2000
Epoch 2/1000
5/5 [==============================] - 2s 593ms/step - loss: 7.2809 - accuracy: 0.2100 - val_loss: 1.6006 - val_accuracy: 0.3700
Epoch 3/1000
5/5 [==============================] - 0s 87ms/step - loss: 1.4362 - accuracy: 0.4600 - val_loss: 1.5722 - val_accuracy: 0.7100
Epoch 4/1000
5/5 [==============================] - 2s 587ms/step - loss: 0.9688 - accuracy: 0.6200 - val_loss: 1.4917 - val_accuracy: 0.7300
Epoch 5/1000
5/5 [==============================] - 0s 78ms/step - loss: 0.7743 - accuracy: 0.6400 - val_loss: 1.5013 - val_accuracy: 0.4200
Epoch 6/1000
5/5 [==============================] - 0s 90ms/step - loss: 0.7229 - accuracy: 0.7300 - val_loss: 1.4708 - val_accuracy: 0.7800
Epoch 7/1000
5/5 [==============================] - 2s 577ms/step - loss: 0.4648 - accuracy: 0.7500 - val_loss: 1.4135 - val_accuracy: 0.7200
Epoch 8

In [29]:
#model_crs.summary()
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

76/76 [==============================] - 6s 48ms/step
array([[396,   5,   6,  20,  31],
       [ 16, 607,  11,  20,   0],
       [ 15,   4, 342,  49,   8],
       [ 23,  11, 232, 297,  18],
       [  0,   2,   1,   2, 309]], dtype=int64)
              precision    recall  f1-score   support

           0       0.88      0.86      0.87       458
           1       0.97      0.93      0.95       654
           2       0.58      0.82      0.68       418
           3       0.77      0.51      0.61       581
           4       0.84      0.98      0.91       314

    accuracy                           0.80      2425
   macro avg       0.81      0.82      0.80      2425
weighted avg       0.82      0.80      0.80      2425



In [30]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=64, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
2/2 [==============================] - 3s 515ms/step - loss: 4.8825 - accuracy: 0.1700 - val_loss: 1.6324 - val_accuracy: 0.2000
Epoch 2/1000
2/2 [==============================] - 2s 63ms/step - loss: 2.7697 - accuracy: 0.2300 - val_loss: 1.5751 - val_accuracy: 0.2600
Epoch 3/1000
2/2 [==============================] - 0s 46ms/step - loss: 1.3456 - accuracy: 0.4100 - val_loss: 1.5428 - val_accuracy: 0.2200
Epoch 4/1000
2/2 [==============================] - 0s 44ms/step - loss: 2.8653 - accuracy: 0.2700 - val_loss: 1.6000 - val_accuracy: 0.2000
Epoch 5/1000
2/2 [==============================] - 0s 59ms/step - loss: 1.5668 - accuracy: 0.2600 - val_loss: 1.5500 - val_accuracy: 0.4800
Epoch 6/1000
2/2 [==============================] - 0s 61ms/step - loss: 0.9165 - accuracy: 0.6600 - val_loss: 1.4299 - val_accuracy: 0.8200
Epoch 7/1000
2/2 [==============================] - 0s 45ms/step - loss: 0.4971 - accuracy: 0.8000 - val_loss: 1.5317 - val_accuracy: 0.6300
Epoch 8/1000

In [31]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

76/76 [==============================] - 3s 5ms/step
array([[381,   5,   2,  11,  59],
       [ 30, 610,   1,   4,   9],
       [ 22,   9, 276,  85,  26],
       [ 40,  73, 238, 190,  40],
       [  0,  10,   1,  10, 293]], dtype=int64)
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       458
           1       0.86      0.93      0.90       654
           2       0.53      0.66      0.59       418
           3       0.63      0.33      0.43       581
           4       0.69      0.93      0.79       314

    accuracy                           0.72      2425
   macro avg       0.70      0.74      0.71      2425
weighted avg       0.72      0.72      0.70      2425



In [32]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=1024,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
2/2 [==============================] - 6s 3s/step - loss: 29.5074 - accuracy: 0.2000 - val_loss: 1.6902 - val_accuracy: 0.2000
Epoch 2/1000
2/2 [==============================] - 0s 106ms/step - loss: 3.2112 - accuracy: 0.1900 - val_loss: 1.6052 - val_accuracy: 0.4000
Epoch 3/1000
2/2 [==============================] - 0s 61ms/step - loss: 1.5156 - accuracy: 0.4100 - val_loss: 1.5875 - val_accuracy: 0.2000
Epoch 4/1000
2/2 [==============================] - 0s 60ms/step - loss: 2.8796 - accuracy: 0.2400 - val_loss: 1.5964 - val_accuracy: 0.2000
Epoch 5/1000
2/2 [==============================] - 0s 106ms/step - loss: 1.2996 - accuracy: 0.4700 - val_loss: 1.4673 - val_accuracy: 0.5400
Epoch 6/1000
2/2 [==============================] - 0s 119ms/step - loss: 0.9365 - accuracy: 0.6000 - val_loss: 1.5702 - val_accuracy: 0.7300
Epoch 7/1000
2/2 [==============================] - 2s 2s/step - loss: 0.7550 - accuracy: 0.7800 - val_loss: 1.4937 - val_accuracy: 0.6300
Epoch 8/1000


In [33]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

76/76 [==============================] - 3s 35ms/step
array([[419,   2,   4,   0,  33],
       [ 44, 593,   8,   0,   9],
       [ 51,   3, 352,   0,  12],
       [159,   9, 384,   0,  29],
       [ 31,   5,   7,   0, 271]], dtype=int64)
              precision    recall  f1-score   support

           0       0.60      0.91      0.72       458
           1       0.97      0.91      0.94       654
           2       0.47      0.84      0.60       418
           3       0.00      0.00      0.00       581
           4       0.77      0.86      0.81       314

    accuracy                           0.67      2425
   macro avg       0.56      0.71      0.61      2425
weighted avg       0.55      0.67      0.60      2425



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
